In [61]:
import os
import pandas as pd
import numpy as np
import re
from scipy.signal import argrelmin, argrelmax
from bisect import bisect_right
from scipy.interpolate import UnivariateSpline

#Plotting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (15, 10)
import seaborn as sns
pd.options.mode.chained_assignment = None

In [161]:
def processData(game_id):
    os.chdir("C:\Users\ckchiruka\Desktop\Current Courses\STA 160\CSV Data")
    motion=pd.read_csv("00"+str(game_id)+".csv",low_memory=False)
    motion=motion[motion.isnull().any(axis=1)==False].reset_index(drop=True)
    return(motion)

def getPlayRanges(motion):
    start = 0
    end = 0
    inplay = []
    no_dups = motion.drop_duplicates(subset = ['quarter', 'sec_in_quarter', 'shot_clock'], keep = 'last')
    nd_index = no_dups.index
    for i in range(max(nd_index)):
        if i not in nd_index:
            start = i
        else:
            inplay.append(start+1)
    
        if i in nd_index:
            end = i  
        else:
            inplay.append(end)

    inplay = sorted(list(set(inplay)))
    inplay = [i for i in inplay if i + 1 in nd_index or i - 1 in nd_index]

    if len(inplay) % 2 == 1:
        inplay.append(len(motion)-1)
    
    inplay = np.asarray(inplay).reshape((len(inplay)/2,2))

    minima = getRelMinima(motion)
    
    for i in range(len(inplay)):
        try:
            inplay[i][0] = minima[bisect_right(minima, inplay[i][0])-1]
        except IndexError:
            pass
        try: 
            inplay[i][1] = minima[bisect_right(minima, inplay[i][1])]
        except IndexError:
            pass
        
    minRanges = []
    for i in inplay:
        temp = [j for j in minima if j >= i[0] and j <= i[1]]
        minRanges.append(temp)
    return minRanges


def getRelMinima(motion):
    minima = list(argrelmin(motion['ball_z'].rolling(3, center = True).mean().as_matrix()))
    minima = [i for i in minima[0]]
    indices = np.where(np.diff(minima)<3)
    indices = [i for i in indices[0]]
    for index in sorted(indices, reverse=True):
        del minima[index]
    return minima

def getRelMaxima(motion):
    maxima = list(argrelmax(motion['ball_z'].rolling(3, center = True).mean().as_matrix()))
    maxima = [i for i in maxima[0]]
    indices = np.where(np.diff(maxima)<3)
    indices = [i for i in indices[0]]
    for index in sorted(indices, reverse=True):
        del maxima[index]
    return maxima

def makeFeatures(minRanges, motion):
    return featuresdf

In [162]:
motion = processData(21500592)  
ranges = getPlayRanges(motion)

In [164]:
for i in ranges:
    for j in range(len(i)-1):
        poly = motion['ball_z'].iloc[i[j]:i[j+1]].as_matrix()
        poly_index = np.arange(0, len(motion['ball_z'].iloc[i[j]:i[j+1]]))
        f = UnivariateSpline(poly_index, poly, k = 2)
        print f.get_coeffs()

[ 10.8904279   13.99920668   3.45988352]
[ 3.48957091  4.43976302  3.87270855]
[ 3.51034775  6.27926049  6.14214975]
[ 5.54241543  7.90962607  1.872476  ]
[ 0.23704739  9.39562039  0.60600271]
[ 0.37114065  8.0694251   1.84933265]
[ 0.26009076  9.29668258  5.2564529 ]
[  4.71775167  12.35140056   2.78819325]
[ 3.59754125  7.93608849  3.25548109]
[ 3.20842132  4.58700389  7.09383077]
[  6.53562739  12.13518336   0.9500536 ]
[ 0.14872564  4.66854732  1.16341673]
[ 0.26529873  5.1249584   3.29663128]
[  2.39050635  14.1365761    8.98492324]
[  9.49712982  14.891886     2.41229121]
[-0.01552902  9.42115802  2.05273193]
[ 0.35843526  8.43364761  1.00523613]
[ 0.55754091  3.096883    3.69762509]
[ 3.28065682  6.17923624  1.18573882]
[ 0.39093559  5.07506651  4.39484009]
[ 4.49930286  4.63118929  4.15089571]
[ 3.85229381  6.61915845  1.27800452]
[ 1.48335338  5.48010265  3.49806453]
[ 3.19208838  7.8822779   3.66749478]
[ 3.79864643  3.94874607  3.80070143]
[ 3.83106543  4.00300771  3.5155214

[ 0.31330388  7.01289476  1.539078  ]
[ 0.38130729  4.81761411  3.17726129]
[ 2.85143221  7.24436485  1.83075475]
[ 0.21969877  6.73923461  1.50399368]
[ 0.3255278   4.84575748  4.17756143]
[ 4.30676708  4.24120396  3.14620625]
[ 3.03500154  4.4757102   0.81202608]
[ 0.62763815  5.66877279  1.33096915]
[ 0.48833119  3.28373655  2.66708333]
[  3.07039543  24.89783886   0.74982206]
[ 0.4312658   5.67666213  3.33428837]
[ 4.00898065  4.92525335  3.41341791]
[ 3.54298167  3.82324542  3.38304   ]
[ 3.41363214  7.37434215  4.09083436]
[ 4.28610495  5.33749484  1.19322231]
[ 0.05953132  6.27463467  0.83293289]
[ 0.59525333  7.53290265  1.63031687]
[ 0.57660603  7.51999029  1.67668387]
[ 0.26643353  5.62214568  4.46158368]
[ 4.49170176  5.35849453  0.7762856 ]
[ 0.38623535  7.78379723  1.67246289]
[-0.1501078   9.21718004  1.22596692]
[ 0.72666565  7.37555042  2.03732115]
[ 0.56574252  7.31568081  1.62013189]
[ 1.19775478  8.48132434  2.82714278]
[ 3.13018874  6.14925709  6.673706  ]
[ 6.41722

[ 4.09563467  5.06096806  4.84681568]
[ 4.66979088  6.20814916  1.76370544]
[ 0.38475465  5.90696014  1.22042237]
[ 0.27838454  6.05570237  1.54196368]
[ 0.2258293   7.82928832  1.15709614]
[ -0.30342518  15.2736679   11.45557466]
[ 10.05154827  12.24509649  11.420373  ]
[ 12.08078305   6.54571      2.03401061]
[ 0.81342978  6.9100029   0.77403978]
[ 0.58778211  7.43776399  1.48812825]
[-0.06133517  8.32227579  1.24491765]
[ 1.06830571  4.37799017  4.84648989]
[ 4.59312639  6.26192897  0.92038754]
[ 0.36943373  8.76472094  1.60312585]
[ 0.03679926  6.73863309  1.29822906]
[ 0.08799385  5.94069454  1.59930235]
[ 0.10494775  6.14391835  0.55761689]
[ 0.0637989   5.14727813  1.61788791]
[ 0.42331296  3.37202397  3.63276096]
[  3.30724186  13.37356856   0.36116478]
[ -0.85000405   8.92004809  18.18320774  12.31638738]
[ 11.0542248   14.77032821   2.60401387]
[ 2.5188636   6.97630232  3.6250668 ]
[ 3.63582126  5.07797176  1.1836549 ]
[ 0.42190923  9.56037977  1.27560677]
[ 0.65339945  4.066

[ 0.55569582  5.17988966  1.20012282]
[ 0.62360376  4.66607774  2.85885426]
[ 1.88796225  8.7974142   8.88801148]
[ 7.9737975   9.84683458  7.92656583]
[ 10.71857177  13.81460703   5.71548791   2.81330581]
[ 3.8839237   5.25251359  5.2716157 ]
[ 5.26222345  4.98219691  3.24031673]
[ 3.03188343  4.82004276  0.66600979]
[ 0.52292634  7.11891374  1.549024  ]
[  0.20306     10.41229798   1.68625368]
[ 0.78126352  6.32903987  0.61661724]
[ 0.10022339  5.60913737  1.71582339]
[ 0.39800377  8.80294882  0.51692223]
[-0.03499975  6.70484689  1.14711339]
[ 0.48430158  7.4839241   1.58452316]
[ 0.45410333  3.869495    3.61177667]
[ 3.56964429  4.19583964  1.17925929]
[-0.07170393  7.76470771  2.0552043 ]
[ 0.22177209  5.38496788  3.40068991]
[  3.38296328  11.09441852  10.84719442]
[ 10.80683844  10.19479111   4.42620026]
[ 4.75426189  7.44704862  4.97551442]
[ 4.83398282  9.9890962   1.48594169]
[ 0.17309215  8.29320479  1.7663028 ]
[ 0.04699815  8.30177834  0.90421854]
[ 0.14748368  7.49740535 

[ 0.28428181  1.84386552  0.55150643]
[ 0.33991091  1.28775561  0.41270636]
[ 0.47491595  1.13949952  0.60406595]
[ 0.36344357  0.80401375  0.66064643]
[ 0.377877  0.781995  0.823773]
[-0.44532005  7.90825256  4.36061184]
[ 3.76993363  5.95961651  6.83267978]
[ 6.67263189  5.37976059  4.32880189]
[ 4.26094953  5.33560284  0.54048299]
[ 0.41444748  6.17032856  1.1859985 ]
[ 0.25669535  8.30885199  1.44249825]
[ 0.15222426  8.97423481  1.98349339]
[  0.61711048  10.44661316   3.08167794]
[ 3.69009424  5.13860589  1.33563424]
[ 0.91338344  9.28740877  2.84167429]
[  2.82136458  10.34110227   2.46321509]
[ 3.00092542  3.42108562  3.37806458]
[ 3.32132964  3.79564961  2.97568164]
[ 2.37863097  9.92884752  3.2435929 ]
[ 3.79789086  3.93687743  3.60380686]
[ 3.54227648  4.1508816   4.71568396]
[  3.38115866  14.68211101   2.36378979]
[ 3.47293002  4.67848198  2.9479415 ]
[ 2.88887231  4.33307481  1.30775308]
[ 0.30485461  6.16852738  0.82829284]
[ 0.672882    6.98837554  0.62761484]
[ 0.62983

...

[ 2.9084275   5.31145375  3.1760725 ]
[ 3.33304745  4.12854675  1.46387012]
[ 1.13453399  4.32860812  1.48585126]
[ 0.14934176  6.13282075  1.07050066]
[ 0.4601386   8.63434406  1.03277246]
[ 0.81446696  5.86648826  1.56929839]
[ 0.05580866  6.96415727  1.15440474]
[ 0.36446599  6.59944667  1.04777754]
[ 0.47495993  4.86054819  0.73692084]
[ 0.25793901  5.69379213  1.12328736]
[ 0.72674007  5.61517957  1.0447985 ]
[ 0.13346279  6.79703509  0.88871364]
[ 0.42243614  6.95847876  0.80511107]
[ 0.32234716  7.60455154  0.91751422]
[ 0.453613    6.50813512  1.19743849]
[ 0.13982167  7.30450139  1.61590328]
[ 0.05245727  6.91860621  1.51679099]
[ 0.50719701  7.8633689   0.99046542]
[ 0.59326804  6.11544133  1.45080503]
[ 0.2876483   4.89471676  1.02907177]
[ 0.0999721   3.82406601  0.75608301]
[ 0.29302781  5.91441156  1.02827762]
[ 0.58235093  4.66392634  3.96906264]
[  3.38227205  16.35931057   0.02236692]
[  1.04256384   3.77744368  10.01178592]
[  7.50034971  10.43654486   9.87270571]
[  

[ 1.692065  2.50727   0.917585]
[ 3.21486664  8.77179584  3.62379795]
[ 4.0942189   5.0336551   1.32952501]
[ 0.41448508  4.44225238  1.60220056]
[ 0.33539531  6.17510579  1.02137183]
[ 0.35410121  4.08864189  1.36290956]
[ 0.3739675  5.0801884  2.920532 ]
[ 2.14731087  7.33271068  7.98960065]
[  7.87038405  13.71085383   0.96353425]
[ -0.99558771  19.73487548   2.10412851]
[  0.51566663  17.32502521  -0.10740623]
[ 0.03871469  6.12510889  1.42450006]
[ 0.50751873  5.49911776  2.61951564]
[  0.84385296  18.89878688  12.20407715]
[ 10.61274018  17.41126336   1.00508388]
[ 0.68240374  6.7431641   3.47578964]
[ 3.06365691  7.40167496  1.30542805]
[ 0.56717386  6.72316304  1.11240186]
[ 0.27029503  9.65166187  1.07182298]
[ 0.59547548  7.4526021   1.3118508 ]
[ 0.06569486  7.13748113  1.23772823]
[ 0.50280969  6.94296564  1.21965424]
[ 0.57674922  5.27230854  0.76010183]
[ 0.30423376  3.51617572  1.28972676]
[ 0.56545396  4.38510284  1.19060843]
[ 0.54359699  4.06287965  2.57554601]
[ 2.11

[ 2.75773671  6.76083625  5.68137621]
[ 5.10114044  9.17795127  1.34811782]
[ 0.09555252  7.64350009  1.41507035]
[ 0.34117664  5.66221255  3.40517578]
[ 3.59030191  4.6253421   0.54349264]
[ 0.54351394  6.29575811  1.16885826]
[ 0.52731764  5.32010143  4.3652447 ]
[ 4.35070301  5.24342283  1.34711119]
[ 0.0370389   7.83418397  1.06215105]
[ 0.40499346  4.56986085  4.06340689]
[ 4.01683879  4.72069385  3.85334275]
[ 3.40556536  5.45317062  6.78311964]
[  3.19208482  13.64545687   2.49688522]
[ 2.911029   3.1412775  2.943171 ]
[  2.17424659  12.96803144   3.38299814]
[ 3.91669318  5.34761781  5.42561845]
[ 5.32272746  3.89121316  1.20125877]
[ 1.11196245  2.28521026  0.56376668]
[-0.06492448  8.70516311  0.61897923]
[ 0.57935005  4.97001754  2.33637725]
[ 2.55650473  2.701879    1.85770327]
[ 2.15479161  4.99379139  0.92260193]
[ 1.44780036  1.47573187  1.56819464]
[ 1.06120482  3.87358264  1.46200632]
[ 0.10023256  5.5593127   1.15574806]
[ -1.00369354  21.76448659   9.02035448]
[ 7.90

[ 0.3726315   3.62341125  2.1599685 ]
[ 2.09242  2.47821  2.27396]
[  1.73278714   5.71976452  12.98678999]
[ 12.68374   12.701955  12.64635 ]
[ 12.97392057  11.01581518   1.38721604]
[ 0.28069462  4.52755788  2.98954626]
[ 2.60412727  6.94690438  3.20504176]
[ 0.3619117   3.72345008  3.5806417 ]
[ 3.09043744  6.35600778  1.22426339]
[ 0.24830512  5.79233278  1.36052012]
[ 0.12099298  7.04483376  1.31354255]
[ 0.11987662  6.1665843   1.21474603]
[ 0.30665719  6.70217323  1.41709424]
[ 0.14892878  7.1711725   1.22493943]
[ 0.6779801   6.57775669  1.26582856]
[ 0.51937888  6.21471389  1.09475586]
[ 0.78823859  8.1592775   2.34217078]
[ 2.8873325   3.73836167  1.48025917]
[ 0.15633536  5.34522295  1.3219925 ]
[ 0.60010126  4.70596618  4.22720326]
[  3.26256017  11.44391913   8.78041061]
[ 9.01604469  8.04716016  0.70947977]
[ 0.67008233  6.28083255  1.35032174]
[ 0.50481667  3.12987625  2.51937833]
[  1.89995525  15.09715845  10.32829634]
[  9.87458804  11.50064978   9.10790661]
[  8.7411

[ 10.22571011  12.02664897   8.39978954]
[  8.22085406  10.28599636   4.76548406]
[ 5.00500617  8.67119334  4.15983552]
[ 4.4714285   4.70438375  4.5530615 ]
[ 4.79350231  4.23484539  1.21154648]
[ 0.24897104  4.51199138  3.49484932]
[ 3.25218126  3.96383206  0.65922762]
[ 0.48030422  5.7803611   1.21868965]
[ 0.67352929  2.10911714  2.25477357]
[ 1.92142091  3.2876816   0.70652758]
[-0.45894976  5.6506968   2.06114349]
[ 0.12104174  8.75495761  1.04323213]
[ 0.40230964  8.6149756   0.94619974]
[ 0.63361608  8.68941112  2.02266168]
[ 0.71519313  7.9367423   0.58405808]
[ 0.58675692  3.91807078  3.73254571]
[ 3.78245218  3.84808727  3.10977085]
[ 3.14767279  3.45068628  4.06739812]
[ 3.97946   3.971495  4.08351 ]
[ 3.48520771  8.32862991  3.87875406]
[ 4.26605027  4.61014376  4.18096645]
[ 4.42810323  3.23375693  0.69849545]
[ 1.16155275  1.31262996  0.96960618]
[ 0.96700415  1.38627688  2.25775865]
[ 2.06406679  2.41802705  1.85554536]
[ 1.84908893  1.74717134  3.27289321]
[ 3.26346079